In [53]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json
import matplotlib.cm as cm

s = "400" #Pick slice around peak for fun

with open(f"exp8_slice{s}_partitions_leiden.json","r") as inff:
    parts = json.load(inff)

with open(f"exp8_slice{s}_graph.json","r") as inff:
    graph = json.load(inff)


removed_nodes = []
reduced_parts = {}
for p in parts.keys():
    pp = parts[p]
    if len(pp) > 100:
        reduced_parts[p] = pp 
    else:
        removed_nodes.append(pp)


removed_nodes = [j for i in removed_nodes for j in i] #one list
removed_nodes = [int(i) for i in removed_nodes]

source = np.array(graph["source"])
target = np.array(graph["target"])

N = len(source)

indices = []
for n in removed_nodes:
    sh = np.where(source == n)[0]
    th = np.where(target == n)[0]
    if len(sh) > 0:
        for ss in sh:
            indices.append(ss)
        for tt in th:
            indices.append(tt)


indices = set(indices)  #remove duplicates
indices = list(indices)  #middle man
indices = np.array(indices)

source = np.delete(source,indices)
target = np.delete(target,indices)

# ------ NX takes "node" : partition_num dictionary :-((

partition = {}
for k in reduced_parts.keys():
    p = reduced_parts[k]
    for n in p:
        partition[str(n)] = int(k)

#print(partition)

list_of_edges = []
for i in range(len(source)):
    list_of_edges.append((str(source[i]), str(target[i])))

G = nx.Graph()
G.add_edges_from(list_of_edges)


pos = nx.kamada_kawai_layout(G)

# color the nodes according to their partition
cmap = cm.get_cmap("viridis", max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=1, cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()
#plt.savefig(self.path_to_plots + "NetworkX_colored_partition_slice" + self.slice_num + ".pdf")
#plt.close()





KeyboardInterrupt: 